In [2]:
import pandas as pd
import plotly.express as px

In [3]:
df_org_data = pd.read_parquet("escooter_copy.parquet")

In [4]:
df_org_data.holiday = df_org_data.holiday.astype(bool)
df_org_data.workingday = df_org_data.workingday.astype(bool)
df_org_data.loc[df_org_data.temp >45, "temp"] = df_org_data.temp.div(100)

In [5]:
df = df_org_data.copy()

In [6]:
df = pd.get_dummies(df, prefix=["w"])

In [59]:
df_w = df.groupby(["registered_customer", pd.Grouper(key="datetime", freq= "W")]).agg(
               humidity= ("humidity", "mean"), 
               windspeed= ("windspeed", "mean"), 
               temp= ("temp", "mean"), 
               atemp= ("atemp", "mean"), 
               reg_customer= ("registered_customer", "count"), 
               holiday= ("holiday", "sum"), 
               workingday= ("workingday", "sum"),  
               clear_few_clouds = ("w_clear, few clouds", "sum"),
               w_cloudy_mist = ("w_cloudy, mist", "sum"),
               w_heacy_rain_thund_snow_icepal= ("w_heacy rain or thunderstorm or snow or ice pallets", "sum"), 
               w_light_snow_rain_thund= ("w_light snow or rain or thunderstorm", "sum")).reset_index()
df_w.head()

,registered_customer,datetime,humidity,windspeed,temp,atemp,reg_customer,holiday,workingday,clear_few_clouds,w_cloudy_mist,w_heacy_rain_thund_snow_icepal,w_light_snow_rain_thund
0,False,2020-01-05,75.681818,16.941545,16.165714,19.566180,462,0,0,122.0,293.0,0,47.0
1,False,2020-01-12,42.836826,15.772793,9.044551,10.791407,668,0,546,547.0,121.0,0,0.0
2,False,2020-01-19,44.462908,15.305623,9.813234,11.712122,674,0,201,475.0,196.0,0,3.0
3,False,2020-01-26,46.770459,14.862526,8.011776,9.945649,501,117,237,259.0,229.0,0,13.0
4,False,2020-02-02,67.263435,7.021275,8.674373,11.812150,949,0,342,591.0,319.0,0,39.0


In [104]:
fig = px.scatter(df_w, x= "datetime", 
              y= "reg_customer", 
              #facet_row= "registered_customer",
              title= "Quantity Comparison of Registered and Unregistered Customers",
              color= "registered_customer", 
              labels= dict(reg_customer="Total Customer", datetime="Datetime", registered_customer="Registered Customer"), 
              marginal_y = "box", 
              trendline= "lowess")

fig.add_vrect(x0="2021-09-2", 
              x1="2022-01-2", 
              col=1,
              annotation_text="decline 2", 
              annotation_position="top left",
              fillcolor="green", 
              opacity=0.2, 
              line_width=0,
              annotation_font_color = "black",
              annotation_font_size = 13)

fig.add_vrect(x0="2020-7-26", 
              x1="2021-01-24", 
              col=1,
              annotation_text="decline 1", 
              annotation_position="top left",
              fillcolor="green", 
              opacity=0.2, 
              line_width=0,
              annotation_font_color = "black",
              annotation_font_size = 13)              

fig.add_hline(y= df_w.loc[df_w.registered_customer == True].reg_customer.mean(), 
              line_dash= "dot", 
              annotation_text= "Reg_True_Cust_Mean",
              annotation_position="bottom left",
              col= 1,
              opacity = 0.3,
              line_color = "red",
              annotation_font_color = "red")

fig.add_hline(y= df_w.loc[df_w.registered_customer == False].reg_customer.mean(), 
              line_dash= "dot", 
              annotation_text= "Reg_False_Cust_Mean",
              annotation_position="bottom left",
              col= 1,
              opacity = 0.3,
              line_color = "blue",
              annotation_font_color = "blue")

fig.update_annotations(opacity= 0.4)
#fig.update_layout(template= "plotly_dark")
fig.update_layout(height= 600, width= 1470)